# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)
---
## 

## CRUD pour verifier les donneés

In [2]:
# import
from datetime import datetime
import pandas as pd
from passlib.hash import argon2



from sqlalchemy import Column, Integer, String, Boolean, DateTime, ForeignKey, create_engine, Table
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy.sql import func

from components.models import Client, DonnePersonnel, Commande, Produit, Promotion, Age, Region

In [3]:
#db_reader.py

engine = create_engine("sqlite:///BD_Ventes_de_jeux_video.bd") 

# session
Session = sessionmaker(bind=engine)
session = Session()

## C - CREATE
- create_client(age_id, region_id)
- create_donne_personnel(login, mot_de_passe_hash)
- create_commande(client_id, produit_id, nb_produit)
- create_promotion(produit_id:int, promotion_percent:int, region_id_promo:int)
- create_produit(.......)


In [4]:
# Function pour creater un Client, Commande
def create_client(age_id:int, region_id:int):
    """Create and persist a new Client in the database.

    Args:
        age_id (int): ID of the age category for the client.
        region_id (int): ID of the region where the client belongs.

    Returns:
        Client: The newly created Client object.

    Raises:
        Exception: If the session commit fails, the transaction is rolled back and the exception is re-raised.
    """
    try:
        client = Client(
            age_id=age_id,
            region_id=region_id
        )
        session.add(client)
        session.commit()
        return client
    except Exception as e:
        session.rollback()
        raise e
    
def create_donne_personnel(login, mot_de_passe_hash):
    """Create and persist a new DonnePersonnel (personal data) record for a client.

    Args:
        login (str): Login username for the personal data.
        mot_de_passe_hash (str): Hashed password.

    Returns:
        DonnePersonnel: The newly created DonnePersonnel object.

    Raises:
        Exception: If the session commit fails, the transaction is rolled back and the exception is re-raised.
    """
    try:
        donne = DonnePersonnel(
            login=login,
            mot_de_passe_hash=mot_de_passe_hash,
            date_suppression=None,
            anonymise=False
        )
        session.add(donne)
        session.commit()
        return donne
    except Exception as e:
        session.rollback()
        raise e

def create_commande(client_id, produit_id, nb_produit):
    """Create and persist a new order (Commande) in the database, optionally applying a promotion.

    The function looks for a promotion associated with the given product — 
    if found, the order will reference the promotion; otherwise, no promotion is applied.

    Args:
        client_id (int): ID of the client placing the order.
        produit_id (int): ID of the product being ordered.
        nb_produit (int): Number of units ordered.

    Returns:
        Commande: The newly created Commande object.

    Raises:
        Exception: If the session commit fails. The session will be rolled back and the exception re-raised.
    """
    try:
        promo = session.query(Promotion).filter(Promotion.produit_id=={produit_id}).first()
        promo_id = promo.promotion_id if promo else 0
        session.add(Commande(
            client_id = client_id,
            produit_id = produit_id,
            nb_produit = nb_produit,
            promotion_id = promo_id
            ))
        session.commit()
    except Exception as e:
        session.rollback()
        raise e
    
def create_promotion(produit_id:int, promotion_percent:int, region_id_promo:int):
    """Create a promotion for a given product and link it to a region.

    Args:
        produit_id (int): ID of the product.
        promotion_percent (int): Discount percent.
        region_id_promo (int): ID of the region for this promotion.

    Raises:
        Exception: If commit fails, the session is rolled back and the exception re-raised.
    """
    try:
        region = session.query(Region).filter(Region.region_id==region_id_promo).first()

        obj = Promotion(
            produit_id = produit_id,
            promotion_percent = promotion_percent
        )
        obj.regions.append(region)
        session.add(obj)
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

# R - READ
- read_table(table_class, limit=None, filter_exp=None) -- jeneral
- read_promo(limit=None, filter_exp=None)
- read_produit
- 

In [ ]:
# Function pour faire un roquet et pour retourner DataFrame
def read_table(table_class, limit=None, filter_exp=None):
    """
    table_class: class SQLAlchemy (par exemple.: Client, Commande)
    filter: opt
    limit: opt

    """
    query = session.query(table_class)
    
    if table_class is Client:
        query.update(
        {Client.date_derniere_utilisation: func.now()},
        synchronize_session=False
        )
        session.commit()

    if filter_exp is not None:
        query = query.filter(filter_exp)
    
    if limit:
        query = query.limit(limit)
    
    df = pd.read_sql(query.statement, session.bind)
    return df

def read_promo(limit=None, filter_exp=None):
    query = session.query(Promotion)
    
    if filter_exp:
        query = query.filter(filter_exp)
    if limit:
        query = query.limit(limit)

    for i in query:
        print(f"Promo {i.promotion_id} ({i.promotion_percent}%) apply to regions:")
        for reg in i.regions:
            print("----------", f"  - {reg.region_id} : {reg.region_nom}")
        print(f"Promo {i.promotion_id} ({i.promotion_percent}%) apply to produit: {i.produit_id})")

    df = pd.read_sql(query.statement, session.bind)
    return df

def read_produit(limit=None, filter_exp=None):
    query = session.query(Produit)

    if filter_exp:
        query = query.filter(filter_exp)
    if limit:
        query = query.limit(limit)
    
    df = pd.read_sql(query.statement, session.bind)

    for i in query:
        age = i.year_cod
        genre = i.genre_cod
        platforme = i.platform_cod
        publisher = i.publisher_cod

    df = pd.read_sql(query.statement, session.bind)
    df["year"] = 



In [8]:
df_client = read_table(Client, limit=5)
df_client

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,1,2,1,2025-12-01 13:31:33,2025-12-01 15:42:34
1,2,0,0,2025-12-01 13:31:33,2025-12-01 15:42:34
2,3,2,2,2025-12-01 13:31:33,2025-12-01 15:42:34
3,4,1,0,2025-12-01 13:31:33,2025-12-01 15:42:34
4,5,3,0,2025-12-01 13:31:33,2025-12-01 15:42:34


In [9]:
df_produit = read_table(Produit, limit=2)
df_produit

,produit_id,name,prix,year_n,platform_cod,genre_cod,publisher_cod
0,1,Wii Sports,35,0,0,0,0
1,2,Super Mario Bros.,146,1,1,1,0


In [ ]:
df_promo = read_promo(limit=5)
df_promo


Promo 1 (70%) apply to regions:
----------   - 1 : NA
Promo 1 (70%) apply to produit: 5511)
Promo 2 (50%) apply to regions:
----------   - 4 : Other
Promo 2 (50%) apply to produit: 6512)
Promo 3 (50%) apply to regions:
----------   - 3 : JP
Promo 3 (50%) apply to produit: 6564)
Promo 4 (10%) apply to regions:
----------   - 2 : EU
Promo 4 (10%) apply to produit: 8839)
Promo 5 (60%) apply to regions:
----------   - 2 : EU
Promo 5 (60%) apply to produit: 936)


In [ ]:
session.close()
engine.dispose()

NameError: name 'session' is not defined